# Ejercicio 1 (conteo de tweets): (1 punto)
Desarrollar un notebook de Jupyter, denominado “tweetCount.ipynb” en el que se utilice como fuente de datos Kafka, y en concreto el topic kafkaTwitter, se establezca una duración de batch de un segundo, y se muestre, cada 5 segundos, el número de tweets recibidos en los últimos 10 segundos. <b>¿Alrededor de qué número (aproximado) se estabiliza el número de tweets que se procesan en el lapso de tiempo indicado (10 segundos)? ¿Tiene sentido? ¿Por qué?</b>

In [1]:
import findspark
findspark.init('/opt/spark')

In [2]:
import os
os.environ['JAVA_HOME'] = '/Library/Java/JavaVirtualMachines/jdk1.8.0_171.jdk/Contents/Home/jre'
os.environ['PYSPARK_SUBMIT_ARGS'] = '--master local[*] --packages org.apache.spark:spark-streaming-kafka-0-8_2.11:2.4.4 pyspark-shell'

In [3]:
import pyspark 
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils

In [4]:
# Podemos ver el application manager en http://192.168.0.105:4040/jobs/.
spark = SparkSession.builder.appName('Tweet Count').getOrCreate()

ssc = StreamingContext(spark.sparkContext, 1)
ssc.checkpoint("checkpoint")

In [5]:
kafkaStream = KafkaUtils.createStream(ssc,zkQuorum='localhost:2181',groupId="group_1" , topics={'kafkaTwitter':2})

In [6]:
# Ventana de 10 segundos.
# Slide de 5.
count_windowed = kafkaStream.countByWindow(10, 5).map(lambda x:('Nº de Tweets tras 5 segundos: %s' % x))
count_windowed.pprint(1)

In [7]:
ssc.start()
# ssc.awaitTermination()

-------------------------------------------
Time: 2020-01-11 18:06:38
-------------------------------------------
Nº de Tweets tras 5 segundos: 2329

-------------------------------------------
Time: 2020-01-11 18:06:43
-------------------------------------------
Nº de Tweets tras 5 segundos: 2329

-------------------------------------------
Time: 2020-01-11 18:06:48
-------------------------------------------
Nº de Tweets tras 5 segundos: 79

-------------------------------------------
Time: 2020-01-11 18:06:53
-------------------------------------------
Nº de Tweets tras 5 segundos: 175

-------------------------------------------
Time: 2020-01-11 18:06:58
-------------------------------------------
Nº de Tweets tras 5 segundos: 200

-------------------------------------------
Time: 2020-01-11 18:07:03
-------------------------------------------
Nº de Tweets tras 5 segundos: 194

-------------------------------------------
Time: 2020-01-11 18:07:08
-----------------------------------

## ¿Alrededor de qué número (aproximado) se estabiliza el número de tweets que se procesan en el lapso de tiempo indicado (10 segundos)? ¿Tiene sentido? ¿Por qué?

Las primeras ventanas no tienen registros porque el job que mete datos en el topic no esta activo.

la 3º ventana recoge los primeros registro. Como calcula el sumatorio de los últimos 10 segundos tiene 0 registros para los primeros segundos muestra un número inusualmente bajo: 33.

A partir de la 5º ventana se estabilizan los registros en ~ 190 registros cada 10 segundos.


Tiene sentido puesto que según se desplace la ventana llega a cubrir ventanas con registros por cada segenudo. Las primeras tienen 0 registros en cada segundo procesado y según avanza tienen registros para cada uno de los segundos (batches de 1s.)

NOTA:
Al ejecutar por segunda vez la traza que aparece en el notebook es distinta. La traza de ahora muestra 2000+ registros porque se ha traido todos los mensajes que había en eñl topic sin comitear.